# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# might need to install geoviews... if so: pip install geoviews

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,holualoa,19.6228,-155.9522,300.06,81,40,5.14,US,1715654192
1,1,klaksvik,62.2266,-6.5890,282.66,99,100,6.24,FO,1715654192
2,2,edinburgh of the seven seas,-37.0676,-12.3116,287.43,65,98,6.56,SH,1715654192
3,3,blackmans bay,-43.0167,147.3167,286.33,54,100,0.89,AU,1715654192
4,4,kiruna,67.8557,20.2251,277.66,87,75,2.06,SE,1715654193


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
cities_plot = city_data_df.hvplot.points(geo=True, 
                                         x='Lng', 
                                         y='Lat', 
                                         size='Humidity', 
                                         tiles='CartoDark',
                                         title='Humidity By City'
                                        )

# Display the map
cities_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = (city_data_df['Humidity'] <= 80)
criteria2 = city_data_df[criteria]


# Drop any rows with null values
city_data_df_cleaned = criteria2.dropna()

# Display sample data
city_data_df_cleaned

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,edinburgh of the seven seas,-37.0676,-12.3116,287.43,65,98,6.56,SH,1715654192
3,3,blackmans bay,-43.0167,147.3167,286.33,54,100,0.89,AU,1715654192
5,5,grytviken,-54.2811,-36.5092,265.89,76,64,4.65,GS,1715654193
6,6,adamstown,-25.0660,-130.1015,294.54,73,4,2.05,PN,1715654193
10,10,arraial do cabo,-22.9661,-42.0278,298.00,67,2,3.40,BR,1715654195
...,...,...,...,...,...,...,...,...,...,...
562,562,crane,31.3974,-102.3501,298.59,27,0,0.39,US,1715654321
563,563,kamloops,50.6665,-120.3192,288.70,66,75,2.06,CA,1715654076
564,564,zhanaozen,43.3000,52.8000,286.10,72,63,4.79,KZ,1715654321
565,565,minab,27.1467,57.0801,303.65,33,0,0.60,IR,1715654321


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
df = pd.DataFrame(city_data_df_cleaned)
df = df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df = df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,edinburgh of the seven seas,SH,-37.0676,-12.3116,65,
3,blackmans bay,AU,-43.0167,147.3167,54,
5,grytviken,GS,-54.2811,-36.5092,76,
6,adamstown,PN,-25.0660,-130.1015,73,
10,arraial do cabo,BR,-22.9661,-42.0278,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = { 
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    res = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = res.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
grytviken - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
la ronge - nearest hotel: La Ronge Hotel and Suites
waitangi - nearest hotel: Hotel Chathams
kununurra - nearest hotel: No hotel found
port lincoln - nearest hotel: Boston Hotel
labe - nearest hotel: GHT (Grand Hotel du Tourisme)
laguna - nearest hotel: Holiday Inn Express & Suites
saint-pierre - nearest hotel: Tropic Hotel
west island - nearest hotel: Cocos Village Bungalows
albany - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
port-aux-francais - nearest hotel: Keravel
guarapari - nearest hotel: Osmar Hotel
saint-jean-de-monts - nearest hotel: L'Espadon
ilam - nearest hotel: Hotel Pokhereli
dubrovnik - nearest hotel: Bellevue
karlskrona - nearest hotel: 

yuza - nearest hotel: 後藤屋旅館
ar rutbah - nearest hotel: No hotel found
katol - nearest hotel: No hotel found
anka - nearest hotel: No hotel found
ghormach - nearest hotel: No hotel found
manaure - nearest hotel: hotel palaima
calama - nearest hotel: Apart Hotel Geotel Calama
arteche - nearest hotel: No hotel found
aasiaat - nearest hotel: SØMA
maintirano - nearest hotel: Hotel Safari
qamdo - nearest hotel: 银杏商务宾馆
paka - nearest hotel: Paka Inn
mil'kovo - nearest hotel: Дрозды
port sudan - nearest hotel: Ranya Hotel
kegen - nearest hotel: Kegen
la sarre - nearest hotel: Motel le Bivouac
dhuusamarreeb - nearest hotel: No hotel found
mogocha - nearest hotel: 7Ka
san pedro pochutla - nearest hotel: Olimpia
soderhamn - nearest hotel: First Hotel Statt
kralendijk - nearest hotel: Hotel Bonaire Inn
itoman - nearest hotel: No hotel found
piti village - nearest hotel: The Cliff Hotel
wickenburg - nearest hotel: Best Western Rancho Grande
dalnegorsk - nearest hotel: Дальнегорск
carnarvon - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
2,edinburgh of the seven seas,SH,-37.0676,-12.3116,65,No hotel found
3,blackmans bay,AU,-43.0167,147.3167,54,Villa Howden
5,grytviken,GS,-54.2811,-36.5092,76,No hotel found
6,adamstown,PN,-25.0660,-130.1015,73,No hotel found
10,arraial do cabo,BR,-22.9661,-42.0278,67,No hotel found
...,...,...,...,...,...,...
562,crane,US,31.3974,-102.3501,27,No hotel found
563,kamloops,CA,50.6665,-120.3192,66,No hotel found
564,zhanaozen,KZ,43.3000,52.8000,72,No hotel found
565,minab,IR,27.1467,57.0801,33,هتل صدف


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotels_plot = hotel_df.hvplot.points(geo=True, 
                                         x='Lng', 
                                         y='Lat', 
                                         c='Hotel Name',
                                         size='Humidity', 
                                         tiles='CartoDark',
                                         title='Hotels by City'
                                        )

# Display the map
hotels_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity)